<a href="https://colab.research.google.com/github/rka0285058-afk/Diabetes_Prediction/blob/main/Diabetes_Prediction_RKA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
import tensorflow as tf
from tensorflow import keras
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import pandas as pd
train_df = pd.read_csv('/content/drive/MyDrive/Diabetes_Prediction/train.csv')
test_df = pd.read_csv('/content/drive/MyDrive/Diabetes_Prediction/test.csv')
submission_df = pd.read_csv('/content/drive/MyDrive/Diabetes_Prediction/sample_submission.csv')

In [21]:
# Save the submission DataFrame to a CSV file
submission_df.to_csv('submission.csv', index=False)

print("Submission file 'submission.csv' generated successfully!")

Submission file 'submission.csv' generated successfully!


In [22]:
display(submission_df)

,id,diagnosed_diabetes
0,700000,1
1,700001,1
2,700002,1
3,700003,0
4,700004,1
...,...,...
299995,999995,1
299996,999996,1
299997,999997,1
299998,999998,1



Developed a machine learning model to predict diabetes using the `train.csv` and `test.csv` datasets, and generate a submission file in the format of `sample_submission.csv`.




Explored the `train_df` and `test_df` to understand their structure, identify missing values, and prepared features for model training. This includes handling categorical variables, scaling numerical features, and potentially creating new features.



For initial understanding of the data, I have displayed the first 5 rows of both `train_df` and `test_df`.



In [4]:
print("Train DataFrame Head:")
print(train_df.head())
print("\nTest DataFrame Head:")
print(test_df.head())

Train DataFrame Head:
   id  age  alcohol_consumption_per_week  physical_activity_minutes_per_week  \
0   0   31                             1                                  45   
1   1   50                             2                                  73   
2   2   32                             3                                 158   
3   3   54                             3                                  77   
4   4   54                             1                                  55   

   diet_score  sleep_hours_per_day  screen_time_hours_per_day   bmi  \
0         7.7                  6.8                        6.1  33.4   
1         5.7                  6.5                        5.8  23.8   
2         8.5                  7.4                        9.1  24.1   
3         4.6                  7.0                        9.2  26.6   
4         5.7                  6.2                        5.1  28.8   

   waist_to_hip_ratio  systolic_bp  ...  gender  ethnicity  education_


 I  printed a concise summary of both `train_df` and `test_df` using the `.info()` method to inspect data types and identify non-null counts, which is crucial for understanding missing values.



In [5]:
print("Train DataFrame Info:")
train_df.info()
print("\nTest DataFrame Info:")
test_df.info()

Train DataFrame Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 700000 entries, 0 to 699999
Data columns (total 26 columns):
 #   Column                              Non-Null Count   Dtype  
---  ------                              --------------   -----  
 0   id                                  700000 non-null  int64  
 1   age                                 700000 non-null  int64  
 2   alcohol_consumption_per_week        700000 non-null  int64  
 3   physical_activity_minutes_per_week  700000 non-null  int64  
 4   diet_score                          700000 non-null  float64
 5   sleep_hours_per_day                 700000 non-null  float64
 6   screen_time_hours_per_day           700000 non-null  float64
 7   bmi                                 700000 non-null  float64
 8   waist_to_hip_ratio                  700000 non-null  float64
 9   systolic_bp                         700000 non-null  int64  
 10  diastolic_bp                        700000 non-null  int64  
 11  hear


To understand the distribution and statistical properties of numerical features, I  generated descriptive statistics for both `train_df` and `test_df` using the `.describe()` method.



In [6]:
print("Train DataFrame Descriptive Statistics:")
print(train_df.describe())
print("\nTest DataFrame Descriptive Statistics:")
print(test_df.describe())

Train DataFrame Descriptive Statistics:
                  id            age  alcohol_consumption_per_week  \
count  700000.000000  700000.000000                 700000.000000   
mean   349999.500000      50.359734                      2.072411   
std    202072.738554      11.655520                      1.048189   
min         0.000000      19.000000                      1.000000   
25%    174999.750000      42.000000                      1.000000   
50%    349999.500000      50.000000                      2.000000   
75%    524999.250000      58.000000                      3.000000   
max    699999.000000      89.000000                      9.000000   

       physical_activity_minutes_per_week     diet_score  sleep_hours_per_day  \
count                       700000.000000  700000.000000        700000.000000   
mean                            80.230803       5.963695             7.002200   
std                             51.195071       1.463336             0.901907   
min           


To identify any missing data that might need handling, I have checked for the total number of missing values in each column for both `train_df` and `test_df`.



In [7]:
print("Missing values in Train DataFrame:")
print(train_df.isnull().sum())
print("\nMissing values in Test DataFrame:")
print(test_df.isnull().sum())

Missing values in Train DataFrame:
id                                    0
age                                   0
alcohol_consumption_per_week          0
physical_activity_minutes_per_week    0
diet_score                            0
sleep_hours_per_day                   0
screen_time_hours_per_day             0
bmi                                   0
waist_to_hip_ratio                    0
systolic_bp                           0
diastolic_bp                          0
heart_rate                            0
cholesterol_total                     0
hdl_cholesterol                       0
ldl_cholesterol                       0
triglycerides                         0
gender                                0
ethnicity                             0
education_level                       0
income_level                          0
smoking_status                        0
employment_status                     0
family_history_diabetes               0
hypertension_history                  0
cardi


To understand the distribution of categorical features and assess the need for encoding, I  identifyied all categorical columns in both `train_df` and `test_df` and then printed their unique values and their respective counts.



In [8]:
categorical_cols_train = train_df.select_dtypes(include=['object']).columns
categorical_cols_test = test_df.select_dtypes(include=['object']).columns

print("Unique values and counts for categorical columns in Train DataFrame:")
for col in categorical_cols_train:
    print(f"\n--- Column: {col} ---")
    print(train_df[col].value_counts())

print("\nUnique values and counts for categorical columns in Test DataFrame:")
for col in categorical_cols_test:
    print(f"\n--- Column: {col} ---")
    print(test_df[col].value_counts())

Unique values and counts for categorical columns in Train DataFrame:

--- Column: gender ---
gender
Female    363237
Male      333085
Other       3678
Name: count, dtype: int64

--- Column: ethnicity ---
ethnicity
White       386153
Hispanic    129984
Black       106301
Asian        60120
Other        17442
Name: count, dtype: int64

--- Column: education_level ---
education_level
Highschool      344145
Graduate        261268
Postgraduate     79642
No formal        14945
Name: count, dtype: int64

--- Column: income_level ---
income_level
Middle          290557
Lower-Middle    178570
Upper-Middle    127836
Low              85803
High             17234
Name: count, dtype: int64

--- Column: smoking_status ---
smoking_status
Never      494448
Current    103363
Former     102189
Name: count, dtype: int64

--- Column: employment_status ---
employment_status
Employed      516170
Retired       115735
Unemployed     49787
Student        18308
Name: count, dtype: int64

Unique values and count


After completion of initial data exploration , the next step was to prepare the features for model training. This involves separating the target variable from the training data and then identifying numerical and categorical features in both training and test datasets. This will enable appropriate preprocessing steps like encoding and scaling.



In [9]:
X = train_df.drop('diagnosed_diabetes', axis=1)
y = train_df['diagnosed_diabetes']

# Identify numerical and categorical columns
numerical_cols = X.select_dtypes(include=['int64', 'float64']).columns
categorical_cols = X.select_dtypes(include=['object']).columns

print(f"Numerical columns: {list(numerical_cols)}")
print(f"Categorical columns: {list(categorical_cols)}")


Numerical columns: ['id', 'age', 'alcohol_consumption_per_week', 'physical_activity_minutes_per_week', 'diet_score', 'sleep_hours_per_day', 'screen_time_hours_per_day', 'bmi', 'waist_to_hip_ratio', 'systolic_bp', 'diastolic_bp', 'heart_rate', 'cholesterol_total', 'hdl_cholesterol', 'ldl_cholesterol', 'triglycerides', 'family_history_diabetes', 'hypertension_history', 'cardiovascular_history']
Categorical columns: ['gender', 'ethnicity', 'education_level', 'income_level', 'smoking_status', 'employment_status']



Now that the numerical and categorical columns have been identified, I  applyied preprocessing steps. For numerical features, I  used `StandardScaler` to scale them. For categorical features, I  used `OneHotEncoder` to convert them into a numerical format, ensuring that the `id` column is dropped as it is not a feature.



In [10]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder

# Create a column transformer for preprocessing
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_cols.drop('id')),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_cols)
    ],
    remainder='passthrough'
)

# Apply preprocessing to training and test data
X_processed = preprocessor.fit_transform(X.drop('id', axis=1))
test_df_processed = preprocessor.transform(test_df.drop('id', axis=1))

print("Preprocessing complete. Shape of processed training data:", X_processed.shape)
print("Shape of processed test data:", test_df_processed.shape)

Preprocessing complete. Shape of processed training data: (700000, 42)
Shape of processed test data: (300000, 42)





Choose a machine learning model known for its robustness, such as a Gradient Boosting model (e.g., XGBoost, LightGBM) or a Random Forest. Train the selected model on the preprocessed training data.



To begin the model selection and training phase, I  imported the necessary libraries, split the preprocessed data into training and validation sets, initialized an `LGBMClassifier` model, and then trained it with the training data.



In [11]:
from sklearn.model_selection import train_test_split
from lightgbm import LGBMClassifier

# Split the preprocessed training data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_processed, y, test_size=0.2, random_state=42)

print("Shape of X_train:", X_train.shape)
print("Shape of X_val:", X_val.shape)
print("Shape of y_train:", y_train.shape)
print("Shape of y_val:", y_val.shape)

# Initialize LGBMClassifier model
model = LGBMClassifier(objective='binary', metric='auc', random_state=42)

# Train the model
model.fit(X_train, y_train)

print("LGBMClassifier model training complete.")

Shape of X_train: (560000, 42)
Shape of X_val: (140000, 42)
Shape of y_train: (560000,)
Shape of y_val: (140000,)
[LightGBM] [Info] Number of positive: 348936, number of negative: 211064
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.109935 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1672
[LightGBM] [Info] Number of data points in the train set: 560000, number of used features: 42
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.623100 -> initscore=0.502727
[LightGBM] [Info] Start training from score 0.502727
LGBMClassifier model training complete.



With the model trained, the next step was to evaluate its performance on the validation set to ensure it generalizes well to unseen data. Calculating the AUC score that will provide a robust metric for this binary classification problem.



In [12]:
from sklearn.metrics import roc_auc_score

# Make predictions on the validation set
y_pred_proba = model.predict_proba(X_val)[:, 1]

# Calculate AUC score
auc_score = roc_auc_score(y_val, y_pred_proba)

print(f"AUC Score on validation set: {auc_score:.4f}")

/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


AUC Score on validation set: 0.7221


In [13]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
import pandas as pd # Ensure pandas is imported

# Create a column transformer for preprocessing
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), [col for col in numerical_cols if col != 'id']),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_cols)
    ],
    remainder='passthrough'
)

# Apply preprocessing to training and test data
X_processed_array = preprocessor.fit_transform(X.drop('id', axis=1))
test_df_processed_array = preprocessor.transform(test_df.drop('id', axis=1))

# Get feature names after preprocessing
feature_names = preprocessor.get_feature_names_out()

# Convert processed arrays back to DataFrame with feature names
X_processed = pd.DataFrame(X_processed_array, columns=feature_names)
test_df_processed = pd.DataFrame(test_df_processed_array, columns=feature_names)

print("Preprocessing complete. Shape of processed training data:", X_processed.shape)
print("Shape of processed test data:", test_df_processed.shape)

Preprocessing complete. Shape of processed training data: (700000, 42)
Shape of processed test data: (300000, 42)



With the preprocessing re-executed and `X_processed` now a DataFrame with feature names, It was needed to re-split the data and re-train the `LGBMClassifier` model using the `X_processed` DataFrame. This ensures the model is trained with feature names, which should resolve the `UserWarning` during prediction.



In [14]:
from sklearn.model_selection import train_test_split
from lightgbm import LGBMClassifier

# Split the preprocessed training data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_processed, y, test_size=0.2, random_state=42)

print("Shape of X_train:", X_train.shape)
print("Shape of X_val:", X_val.shape)
print("Shape of y_train:", y_train.shape)
print("Shape of y_val:", y_val.shape)

# Initialize LGBMClassifier model
model = LGBMClassifier(objective='binary', metric='auc', random_state=42)

# Train the model
model.fit(X_train, y_train)

print("LGBMClassifier model training complete.")

Shape of X_train: (560000, 42)
Shape of X_val: (140000, 42)
Shape of y_train: (560000,)
Shape of y_val: (140000,)
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 348936, number of negative: 211064
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.213807 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1672
[LightGBM] [Info] Number of data points in the train set: 560000, number of used features: 42
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.623100 -> initscore=0.502727
[LightGBM] [Info] Start training from score 0.502727
LGBMClassifier model training complete.



Now that the model has been re-trained with feature names, I re-evaluated its performance on the validation set by calculating the AUC score. This step will verify if the `UserWarning` regarding feature names has been resolved and provide the updated model performance metric.



In [15]:
from sklearn.metrics import roc_auc_score

# Make predictions on the validation set
y_pred_proba = model.predict_proba(X_val)[:, 1]

# Calculate AUC score
auc_score = roc_auc_score(y_val, y_pred_proba)

print(f"AUC Score on validation set: {auc_score:.4f}")

AUC Score on validation set: 0.7221





Assess the trained model's performance using additional appropriate metrics (e.g., accuracy, precision, recall, F1-score) on the validation set.



To fully assess the model's performance beyond AUC, I  predicted class labels on the validation set and then calculate and print common classification metrics such as accuracy, precision, recall, and F1-score.



In [16]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Make class predictions on the validation set
y_pred = model.predict(X_val)

# Calculate additional metrics
accuracy = accuracy_score(y_val, y_pred)
precision = precision_score(y_val, y_pred)
recall = recall_score(y_val, y_pred)
f1 = f1_score(y_val, y_pred)

# Print the metrics
print(f"Accuracy on validation set: {accuracy:.4f}")
print(f"Precision on validation set: {precision:.4f}")
print(f"Recall on validation set: {recall:.4f}")
print(f"F1-Score on validation set: {f1:.4f}")

Accuracy on validation set: 0.6807
Precision on validation set: 0.7021
Recall on validation set: 0.8481
F1-Score on validation set: 0.7683





Using the trained model to make predictions on the preprocessed `test_df` and format these predictions into the `submission_df` as required for final submission.






*   The `train_df` contained 700,000 entries and 26 columns (including the target variable `diagnosed_diabetes`), while `test_df` had 300,000 entries and 25 columns.
*   No missing values were found in any column of either the training or test datasets, simplifying the preprocessing step.
*   Categorical features, such as `gender`, `ethnicity`, `education_level`, `income_level`, `smoking_status`, and `employment_status`, were successfully identified and their unique values confirmed.
*   Numerical features were scaled using `StandardScaler`, and categorical features were one-hot encoded using `OneHotEncoder`. The processed training data (`X_processed`) and test data (`test_df_processed`) both resulted in 42 features.
*   An `LGBMClassifier` model was trained on the preprocessed data, achieving an AUC score of 0.7221 on the validation set.
*   Additional evaluation metrics on the validation set showed an Accuracy of 0.6807, Precision of 0.7021, Recall of 0.8481, and an F1-Score of 0.7683.

